In [1]:
import pandas as pd

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader

In [3]:
from utils_training import Dataset_LV
from utils_training import LVTrainingData
from utils_training import evaluate_LVmodel

In [4]:
from behavior_model import LVBehaviorModel

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
# Replace 'your_file.csv' with the actual path to your CSV file
file_path = 'Synthetic_crash_scenarios.csv'
# file_path = '/content/drive/MyDrive/Colab Notebooks/Thesis Crash Generation/Synthetic_crash_scenarios.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

## Build the dataset

In [7]:
acc_min, acc_max, acc_interval =  -9.5, 7, 0.25
acc_std_dev = 0.07
Inputs, Outputs, Weights = LVTrainingData(df)

In [8]:
## Load the train, val, test indexes

import pickle
with open('.\\utils_data\\Idx_dtw_02_split.pkl', 'rb') as f:
    list_train_val_test = pickle.load(f)
list_train_val_test = list_train_val_test[0]

Idx_train, Idx_val, Idx_test = list_train_val_test[0], list_train_val_test[1], list_train_val_test[2]

In [9]:
## Split the Inputs, Outputs, Weights based on train, val, test indexes

Inputs_train = [Inputs[i] for i in Idx_train]
Outputs_train = [Outputs[i] for i in Idx_train]
Weights_train = [Weights[i] for i in Idx_train]

Inputs_val = [Inputs[i] for i in Idx_val]
Outputs_val = [Outputs[i] for i in Idx_val]
Weights_val = [Weights[i] for i in Idx_val]

In [10]:
## Create dataset and dataloader

input_size = Inputs[0].shape[1] # Dimension of the input
batch_size = 64

# dataset = BCDataset(Inputs, Outputs, Weights)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset_train = Dataset_LV(Inputs_train, Outputs_train, Weights_train)
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

dataset_val = Dataset_LV(Inputs_val, Outputs_val, Weights_val)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)

## Model Training

In [11]:
## Network Parameters 
gru_hidden_size = 256 # Size of GRU's hidden layer
mlp_hidden_size = 128 # Size of MLP's hidden layer
output_size = int((acc_max - acc_min)/acc_interval) + 1

## Training Parameters
num_epochs = 500
learning_rate = 0.0001

## Instantiate model, loss function, and optimizer
model = LVBehaviorModel(input_size, gru_hidden_size, mlp_hidden_size, output_size).to(device)
# criterion = nn.KLDivLoss(reduction='batchmean').to(device)
criterion = nn.KLDivLoss(reduction='none').to(device)
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [12]:
best_val_loss = float('inf')
best_train_loss = float('inf')
best_trainval_loss = float('inf')
best_val_epoch = 0
best_train_epoch = 0
best_trainval_epoch = 0
best_val_model_path = f'.\\weights\\bc_lv_val_dtw_02.pth'
best_train_model_path = f'.\\weights\\bc_lv_train_dtw_02.pth'
best_trainval_model_path = f'.\\weights\\bc_lv_trainval_dtw_02.pth'

In [13]:
num_epochs = 4000

In [14]:
for epoch in range(num_epochs):
    model.train() # set model as training mode
    for inputs, targets, sample_weights in dataloader_train:
        inputs = inputs.float().to(device)
        targets = targets.float().to(device)
        sample_weights = sample_weights.to(device)
        
        optimizer.zero_grad()
        log_probs = model(inputs)
        ## calculate loss
        loss = criterion(log_probs, targets)
        weighted_loss = torch.sum(torch.sum(loss,axis=2) * sample_weights.reshape(-1,1))/torch.sum(sample_weights) ## loss per weight
        weighted_loss.backward()
        optimizer.step()
    
    ## Calculate the mean, min, max loss of this epoch
    model.eval()  # set model as evaluation mode
    ## loss of training dataset
    train_loss_mean, train_loss_min, train_loss_max = evaluate_LVmodel(model, criterion, dataloader_train, device)
    ## loss of validation dataset
    test_loss_mean, test_loss_min, test_loss_max = evaluate_LVmodel(model, criterion, dataloader_val, device)
    ## loss of combination
    trainval_loss_mean = (test_loss_mean*len(dataset_val) + train_loss_mean*len(dataset_train)) / (len(dataset_val)+len(dataset_train))

    print(f'Epoch {epoch+1}/{num_epochs}, Loss_mean: {round(train_loss_mean.item(),2)}, \
     Loss_min: {round(train_loss_min.item(),2)}, Loss_max: {round(train_loss_max.item(),2)} \n \
     Loss_mean_val: {round(test_loss_mean.item(),2)}, Loss_min_val: {round(test_loss_min.item(),2)}, \
     Loss_max_val: {round(test_loss_max.item(),2)} \n \
     Loss_mean_trainval: {round(trainval_loss_mean.item(),2)}')

    # Save the model if the current test loss is the best seen so far
    if test_loss_mean < best_val_loss:
        best_val_epoch = epoch+1
        best_val_loss = test_loss_mean
        best_val_model = model.state_dict()
    if train_loss_mean < best_train_loss:
        best_train_epoch = epoch+1
        best_train_loss = train_loss_mean
        best_train_model = model.state_dict()
    if trainval_loss_mean < best_trainval_loss:
        best_trainval_epoch = epoch+1
        best_trainval_loss = trainval_loss_mean
        best_trainval_model = model.state_dict()
        
torch.save(best_val_model, best_val_model_path)
print(f"Model saved with the best val loss. \n Epoch: {best_val_epoch}, Loss_val: {round(best_val_loss.item(),2)}")
torch.save(best_train_model, best_train_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_train_epoch}, Loss_val: {round(best_train_loss.item(),2)}")
torch.save(best_trainval_model, best_trainval_model_path)
print(f"Model saved with the best train loss. \n Epoch: {best_trainval_epoch}, Loss_val: {round(best_trainval_loss.item(),2)}")

Epoch 1/4000, Loss_mean: 393.77,      Loss_min: 391.32, Loss_max: 396.23 
      Loss_mean_val: 391.07, Loss_min_val: 374.79,      Loss_max_val: 399.91 
      Loss_mean_trainval: 391.21
Epoch 2/4000, Loss_mean: 391.8,      Loss_min: 389.93, Loss_max: 393.66 
      Loss_mean_val: 389.32, Loss_min_val: 380.01,      Loss_max_val: 400.71 
      Loss_mean_trainval: 389.45
Epoch 3/4000, Loss_mean: 389.16,      Loss_min: 389.01, Loss_max: 389.31 
      Loss_mean_val: 387.4, Loss_min_val: 379.67,      Loss_max_val: 395.12 
      Loss_mean_trainval: 387.48
Epoch 4/4000, Loss_mean: 386.63,      Loss_min: 386.57, Loss_max: 386.68 
      Loss_mean_val: 385.49, Loss_min_val: 375.94,      Loss_max_val: 394.01 
      Loss_mean_trainval: 385.55
Epoch 5/4000, Loss_mean: 384.24,      Loss_min: 382.87, Loss_max: 385.62 
      Loss_mean_val: 383.75, Loss_min_val: 378.84,      Loss_max_val: 393.49 
      Loss_mean_trainval: 383.77
Epoch 6/4000, Loss_mean: 381.88,      Loss_min: 381.22, Loss_max: 382.54 
   

Epoch 46/4000, Loss_mean: 220.76,      Loss_min: 185.63, Loss_max: 255.89 
      Loss_mean_val: 261.96, Loss_min_val: 179.52,      Loss_max_val: 331.3 
      Loss_mean_trainval: 259.92
Epoch 47/4000, Loss_mean: 217.63,      Loss_min: 211.28, Loss_max: 223.97 
      Loss_mean_val: 261.93, Loss_min_val: 174.15,      Loss_max_val: 338.3 
      Loss_mean_trainval: 259.73
Epoch 48/4000, Loss_mean: 217.41,      Loss_min: 199.59, Loss_max: 235.23 
      Loss_mean_val: 260.26, Loss_min_val: 184.98,      Loss_max_val: 358.86 
      Loss_mean_trainval: 258.14
Epoch 49/4000, Loss_mean: 214.53,      Loss_min: 191.19, Loss_max: 237.87 
      Loss_mean_val: 259.84, Loss_min_val: 158.45,      Loss_max_val: 341.31 
      Loss_mean_trainval: 257.59
Epoch 50/4000, Loss_mean: 212.46,      Loss_min: 206.71, Loss_max: 218.21 
      Loss_mean_val: 256.28, Loss_min_val: 169.34,      Loss_max_val: 338.31 
      Loss_mean_trainval: 254.11
Epoch 51/4000, Loss_mean: 211.46,      Loss_min: 210.67, Loss_max: 212.2

Epoch 91/4000, Loss_mean: 181.31,      Loss_min: 179.83, Loss_max: 182.79 
      Loss_mean_val: 235.94, Loss_min_val: 121.92,      Loss_max_val: 321.76 
      Loss_mean_trainval: 233.23
Epoch 92/4000, Loss_mean: 186.82,      Loss_min: 165.33, Loss_max: 208.32 
      Loss_mean_val: 233.91, Loss_min_val: 148.79,      Loss_max_val: 330.75 
      Loss_mean_trainval: 231.58
Epoch 93/4000, Loss_mean: 180.1,      Loss_min: 178.78, Loss_max: 181.42 
      Loss_mean_val: 232.52, Loss_min_val: 142.84,      Loss_max_val: 326.94 
      Loss_mean_trainval: 229.92
Epoch 94/4000, Loss_mean: 177.98,      Loss_min: 168.9, Loss_max: 187.05 
      Loss_mean_val: 234.1, Loss_min_val: 128.28,      Loss_max_val: 323.54 
      Loss_mean_trainval: 231.32
Epoch 95/4000, Loss_mean: 175.9,      Loss_min: 166.75, Loss_max: 185.05 
      Loss_mean_val: 231.41, Loss_min_val: 144.36,      Loss_max_val: 343.17 
      Loss_mean_trainval: 228.66
Epoch 96/4000, Loss_mean: 178.42,      Loss_min: 174.39, Loss_max: 182.44 

Epoch 136/4000, Loss_mean: 160.38,      Loss_min: 157.67, Loss_max: 163.08 
      Loss_mean_val: 219.94, Loss_min_val: 130.2,      Loss_max_val: 286.91 
      Loss_mean_trainval: 216.98
Epoch 137/4000, Loss_mean: 156.74,      Loss_min: 124.56, Loss_max: 188.92 
      Loss_mean_val: 223.12, Loss_min_val: 133.8,      Loss_max_val: 305.44 
      Loss_mean_trainval: 219.83
Epoch 138/4000, Loss_mean: 160.04,      Loss_min: 159.02, Loss_max: 161.06 
      Loss_mean_val: 219.36, Loss_min_val: 113.31,      Loss_max_val: 309.43 
      Loss_mean_trainval: 216.42
Epoch 139/4000, Loss_mean: 159.78,      Loss_min: 153.89, Loss_max: 165.67 
      Loss_mean_val: 222.23, Loss_min_val: 131.23,      Loss_max_val: 339.11 
      Loss_mean_trainval: 219.13
Epoch 140/4000, Loss_mean: 159.49,      Loss_min: 152.47, Loss_max: 166.5 
      Loss_mean_val: 223.44, Loss_min_val: 129.36,      Loss_max_val: 321.73 
      Loss_mean_trainval: 220.27
Epoch 141/4000, Loss_mean: 159.48,      Loss_min: 149.68, Loss_max: 

Epoch 181/4000, Loss_mean: 145.04,      Loss_min: 138.22, Loss_max: 151.85 
      Loss_mean_val: 212.06, Loss_min_val: 90.67,      Loss_max_val: 330.09 
      Loss_mean_trainval: 208.73
Epoch 182/4000, Loss_mean: 149.9,      Loss_min: 120.21, Loss_max: 179.59 
      Loss_mean_val: 211.14, Loss_min_val: 142.82,      Loss_max_val: 322.44 
      Loss_mean_trainval: 208.1
Epoch 183/4000, Loss_mean: 143.0,      Loss_min: 137.8, Loss_max: 148.2 
      Loss_mean_val: 216.49, Loss_min_val: 115.1,      Loss_max_val: 315.35 
      Loss_mean_trainval: 212.84
Epoch 184/4000, Loss_mean: 143.02,      Loss_min: 141.25, Loss_max: 144.79 
      Loss_mean_val: 212.28, Loss_min_val: 91.3,      Loss_max_val: 311.69 
      Loss_mean_trainval: 208.85
Epoch 185/4000, Loss_mean: 142.49,      Loss_min: 141.63, Loss_max: 143.35 
      Loss_mean_val: 213.09, Loss_min_val: 89.0,      Loss_max_val: 268.29 
      Loss_mean_trainval: 209.59
Epoch 186/4000, Loss_mean: 137.98,      Loss_min: 122.98, Loss_max: 152.98 


...

Epoch 3813/4000, Loss_mean: 4.92,      Loss_min: 4.56, Loss_max: 5.27 
      Loss_mean_val: 272.34, Loss_min_val: 81.75,      Loss_max_val: 539.21 
      Loss_mean_trainval: 259.08
Epoch 3814/4000, Loss_mean: 4.9,      Loss_min: 4.61, Loss_max: 5.2 
      Loss_mean_val: 264.85, Loss_min_val: 137.58,      Loss_max_val: 412.86 
      Loss_mean_trainval: 251.96
Epoch 3815/4000, Loss_mean: 4.95,      Loss_min: 4.4, Loss_max: 5.49 
      Loss_mean_val: 261.89, Loss_min_val: 99.06,      Loss_max_val: 480.9 
      Loss_mean_trainval: 249.15
Epoch 3816/4000, Loss_mean: 4.96,      Loss_min: 4.47, Loss_max: 5.46 
      Loss_mean_val: 265.71, Loss_min_val: 163.12,      Loss_max_val: 451.72 
      Loss_mean_trainval: 252.78
Epoch 3817/4000, Loss_mean: 4.88,      Loss_min: 4.65, Loss_max: 5.11 
      Loss_mean_val: 266.66, Loss_min_val: 106.33,      Loss_max_val: 494.04 
      Loss_mean_trainval: 253.68
Epoch 3818/4000, Loss_mean: 4.84,      Loss_min: 4.17, Loss_max: 5.5 
      Loss_mean_val: 266.6

Epoch 3859/4000, Loss_mean: 4.86,      Loss_min: 4.74, Loss_max: 4.99 
      Loss_mean_val: 268.96, Loss_min_val: 115.98,      Loss_max_val: 500.84 
      Loss_mean_trainval: 255.87
Epoch 3860/4000, Loss_mean: 4.82,      Loss_min: 4.68, Loss_max: 4.96 
      Loss_mean_val: 261.91, Loss_min_val: 138.09,      Loss_max_val: 538.33 
      Loss_mean_trainval: 249.16
Epoch 3861/4000, Loss_mean: 4.87,      Loss_min: 4.81, Loss_max: 4.92 
      Loss_mean_val: 265.08, Loss_min_val: 95.34,      Loss_max_val: 463.3 
      Loss_mean_trainval: 252.18
Epoch 3862/4000, Loss_mean: 5.04,      Loss_min: 4.85, Loss_max: 5.24 
      Loss_mean_val: 265.43, Loss_min_val: 127.35,      Loss_max_val: 411.98 
      Loss_mean_trainval: 252.52
Epoch 3863/4000, Loss_mean: 5.06,      Loss_min: 4.33, Loss_max: 5.79 
      Loss_mean_val: 271.67, Loss_min_val: 128.79,      Loss_max_val: 454.22 
      Loss_mean_trainval: 258.45
Epoch 3864/4000, Loss_mean: 5.12,      Loss_min: 4.19, Loss_max: 6.04 
      Loss_mean_val: 

Epoch 3905/4000, Loss_mean: 5.03,      Loss_min: 4.86, Loss_max: 5.19 
      Loss_mean_val: 266.18, Loss_min_val: 98.62,      Loss_max_val: 473.81 
      Loss_mean_trainval: 253.24
Epoch 3906/4000, Loss_mean: 4.87,      Loss_min: 4.39, Loss_max: 5.35 
      Loss_mean_val: 268.57, Loss_min_val: 167.72,      Loss_max_val: 493.78 
      Loss_mean_trainval: 255.5
Epoch 3907/4000, Loss_mean: 4.89,      Loss_min: 3.86, Loss_max: 5.92 
      Loss_mean_val: 274.49, Loss_min_val: 135.48,      Loss_max_val: 546.23 
      Loss_mean_trainval: 261.12
Epoch 3908/4000, Loss_mean: 4.76,      Loss_min: 4.62, Loss_max: 4.9 
      Loss_mean_val: 268.38, Loss_min_val: 144.85,      Loss_max_val: 446.42 
      Loss_mean_trainval: 255.31
Epoch 3909/4000, Loss_mean: 4.74,      Loss_min: 4.68, Loss_max: 4.79 
      Loss_mean_val: 269.86, Loss_min_val: 97.81,      Loss_max_val: 537.74 
      Loss_mean_trainval: 256.71
Epoch 3910/4000, Loss_mean: 4.77,      Loss_min: 4.21, Loss_max: 5.33 
      Loss_mean_val: 26

Epoch 3951/4000, Loss_mean: 4.97,      Loss_min: 4.53, Loss_max: 5.42 
      Loss_mean_val: 269.1, Loss_min_val: 90.46,      Loss_max_val: 602.07 
      Loss_mean_trainval: 256.0
Epoch 3952/4000, Loss_mean: 5.04,      Loss_min: 4.31, Loss_max: 5.77 
      Loss_mean_val: 263.4, Loss_min_val: 123.3,      Loss_max_val: 458.82 
      Loss_mean_trainval: 250.6
Epoch 3953/4000, Loss_mean: 5.58,      Loss_min: 4.65, Loss_max: 6.52 
      Loss_mean_val: 272.25, Loss_min_val: 98.63,      Loss_max_val: 552.36 
      Loss_mean_trainval: 259.03
Epoch 3954/4000, Loss_mean: 5.0,      Loss_min: 4.46, Loss_max: 5.54 
      Loss_mean_val: 269.94, Loss_min_val: 136.36,      Loss_max_val: 472.1 
      Loss_mean_trainval: 256.8
Epoch 3955/4000, Loss_mean: 5.19,      Loss_min: 5.17, Loss_max: 5.21 
      Loss_mean_val: 262.7, Loss_min_val: 71.52,      Loss_max_val: 510.57 
      Loss_mean_trainval: 249.94
Epoch 3956/4000, Loss_mean: 4.9,      Loss_min: 4.72, Loss_max: 5.08 
      Loss_mean_val: 269.59, Los

Epoch 3997/4000, Loss_mean: 4.61,      Loss_min: 3.99, Loss_max: 5.24 
      Loss_mean_val: 270.56, Loss_min_val: 128.31,      Loss_max_val: 475.17 
      Loss_mean_trainval: 257.37
Epoch 3998/4000, Loss_mean: 4.6,      Loss_min: 3.73, Loss_max: 5.47 
      Loss_mean_val: 268.78, Loss_min_val: 119.82,      Loss_max_val: 444.08 
      Loss_mean_trainval: 255.68
Epoch 3999/4000, Loss_mean: 4.54,      Loss_min: 4.48, Loss_max: 4.61 
      Loss_mean_val: 259.48, Loss_min_val: 99.2,      Loss_max_val: 535.2 
      Loss_mean_trainval: 246.84
Epoch 4000/4000, Loss_mean: 4.57,      Loss_min: 4.49, Loss_max: 4.64 
      Loss_mean_val: 268.08, Loss_min_val: 141.18,      Loss_max_val: 501.53 
      Loss_mean_trainval: 255.02
Model saved with the best val loss. 
 Epoch: 805, Loss_val: 174.08
Model saved with the best train loss. 
 Epoch: 3992, Loss_val: 4.52
Model saved with the best train loss. 
 Epoch: 953, Loss_val: 167.51
